# II. 영화 리뷰 데이터 전처리(Preprocess)

---
### 1) 데이터 로드 : dataframe (pandas module 사용)

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc # garbage collector : 메모리 관리
gc.collect()

In [ ]:
import os

전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'

TRAIN_DATA_FILE = 'ratings_train.txt'
# TEST_DATA_FILE = 'ratings_test.txt'

TRAIN_DATA_PATH = os.path.join(DATA_DIR, TRAIN_DATA_FILE)
# TEST_DATA_PATH = os.path.join(DATA_DIR, TEST_DATA_FILE)

In [4]:
import pandas as pd
import numpy as np

In [5]:
# pandas data frame
train_df = pd.read_table(TRAIN_DATA_PATH) # pd.read_csv(TRAIN_DATA_PATH, sep='\t') 동일한 기능
# test_df = pd.read_table(TEST_DATA_PATH)

---
### 2) 결측 데이터 처리 : nul 데이터는 필요없으므로 먼저 제거한다.

In [6]:
train_no_nan_df = train_df.dropna(axis=0) # nan이 있는 모든 row를 없애는 코드
# test_no_nan_df = test_df.dropna(axis=0)

---
### 3) 한글 외의 문자는 모두 제거

In [7]:
# 정규표현식 : Regular Expression
import re

In [8]:
hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자

In [9]:
# 정규 표현식 테스트
txt = '아 더빙.. 진짜 짜증나네요... 목소리 aa'
txt_kor = hangul.sub('', txt) # 한글만 추출
txt_kor

'아 더빙 진짜 짜증나네요 목소리 '

In [10]:
train_no_nan_df['document_kor'] = train_no_nan_df['document'].apply(lambda x: hangul.sub('', x).strip()).copy() # 빈 칸이 하나 남는 것을 없애기 위해 .strip()을 사용
train_no_nan_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,document,label,document_kor
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 솔직히 재미는 없다평점 조정
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...


In [11]:
train_no_nan_df['document_kor'].replace('', np.nan, inplace=True) # 한글만 남긴 데이터에서 빈 문자열을 NaN값으로 변환한다. 이렇게 하여야 삭제하기 쉽다.

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


---
한글만 남긴 후 생긴 결측 데이터 확인

In [12]:
print('null data가 있는가? : ', train_no_nan_df.isnull().values.any())
print('----------')
print('null data 건수 \n', '-----\n', train_no_nan_df.isnull().sum())

null data가 있는가? :  True
----------
null data 건수 
 -----
 id                 0
document           0
label              0
document_kor    1255
dtype: int64


In [13]:
train_no_nan_df.shape

(149994, 4)

In [14]:
train_no_nan_df = train_no_nan_df.dropna(axis=0) # nan이 있는 모든 row를 없애는 코드

In [15]:
train_no_nan_df.shape

(148739, 4)

In [16]:
# 149995 - 148739

---
### 4) 중복 데이터 제거

In [17]:
train_no_nan_df.shape

(148739, 4)

In [18]:
train_no_nan_df['document'].nunique()

145392

In [19]:
train_no_nan_df['document_kor'].nunique()

143460

In [20]:
145393 - 143461

1932

In [21]:
# 중복된 데이터 확인
train_no_nan_df[train_no_nan_df.duplicated('document_kor')]

,id,document,label,document_kor
1097,2062443,쵝오,1,쵝오
1165,7508856,최고!!,1,최고
1248,3502685,최고,1,최고
1483,775332,최고,1,최고
1531,2196647,재밌군..,1,재밌군
...,...,...,...,...
149949,3924716,OO 영화,0,영화
149960,6368431,지루하다,0,지루하다
149979,4059827,배우들이 아깝다,0,배우들이 아깝다
149987,7669621,재미있어요^^,1,재미있어요


In [22]:
# train_data.drop_duplicates(subset=['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
dedupe_train_df = train_no_nan_df.drop_duplicates('document_kor', keep='first')

In [23]:
dedupe_train_df.shape

(143460, 4)

---
### 5) 형태소 분석
  - 문어체 한국어는 띄어 쓰기에 민감하지 않다.
  - 띄어쓰기만으로 토큰화가 정확하게 되기 어렵다.
  - 문어체 한국어를 토큰화하기 위해서는 형태소 분석이 필요하다.

In [24]:
# !pip install kiwipiepy

In [25]:
# 데이터 확인
dedupe_train_df['document_kor']

0                                         아 더빙 진짜 짜증나네요 목소리
1                                흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                 교도소 이야기구먼 솔직히 재미는 없다평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
                                ...                        
149993                             칼 세이건으로 시작해서 칼 세이건으로 끝난다
149994                  디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요
149995                                      인간이 문제지 소는 뭔죄인가
149997                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998                           청춘 영화의 최고봉방황과 우울했던 날들의 자화상
Name: document_kor, Length: 143460, dtype: object

---
kiwi를 이용한 한국어 형태소 분석  
https://github.com/bab2min/kiwipiepy

In [26]:
# 한국어 형태소 분석을 위한 모듈 임포트
from kiwipiepy import Kiwi

In [27]:
kiwi = Kiwi()
kiwi.prepare()

0

---
형태소 분석 결과 참고 : 품사 태그  
https://github.com/bab2min/kiwipiepy#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8  

In [28]:
# kiwi 형태소 분석 예
kiwi.analyze(txt_kor)

[([('아', 'IC', 0, 1),
   ('더빙', 'NNG', 2, 2),
   ('진짜', 'MAG', 5, 2),
   ('짜증', 'NNG', 8, 2),
   ('나', 'VV', 10, 1),
   ('네요', 'EF', 11, 2),
   ('목소리', 'NNG', 14, 3)],
  -63.02336502075195)]

In [29]:
 dedupe_train_df['document_kor']

0                                         아 더빙 진짜 짜증나네요 목소리
1                                흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                                 교도소 이야기구먼 솔직히 재미는 없다평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...
                                ...                        
149993                             칼 세이건으로 시작해서 칼 세이건으로 끝난다
149994                  디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요
149995                                      인간이 문제지 소는 뭔죄인가
149997                        이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다
149998                           청춘 영화의 최고봉방황과 우울했던 날들의 자화상
Name: document_kor, Length: 143460, dtype: object

In [30]:
import time

In [31]:
start = time.time() # start time

dedupe_train_df['morphed'] = dedupe_train_df['document_kor'].apply(lambda x: kiwi.analyze(x)[0]) # 2 분 정도 걸림. # data 중에 형식이 맞지 않는 데이터가 입력되면 에러가 난다. NaN이 있는 경우
# df_new = dedupe_kor_df.apply(lambda x: (kiwi.analyze(x))[0])

end = time.time() # end time

print("time : ", end - start, " sec") # sec

time :  112.736412525177  sec


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
dedupe_train_df

,id,document,label,document_kor,morphed
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,아 더빙 진짜 짜증나네요 목소리,"([(아, IC, 0, 1), (더빙, NNG, 2, 2), (진짜, MAG, 5,..."
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,"([(흠, IC, 0, 1), (포스터, NNG, 1, 3), (보, VV, 4, ..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,너무재밓었다그래서보는것을추천한다,"([(너무재밓었다, NNG, 0, 6), (그래서, MAJ, 6, 3), (보, V..."
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,교도소 이야기구먼 솔직히 재미는 없다평점 조정,"([(교도소, NNG, 0, 3), (이야기, NNG, 4, 3), (구먼, EC,..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,"([(사이몬페그, NNP, 0, 5), (의, JKG, 5, 1), (익살, NNG..."
...,...,...,...,...,...
149993,10020916,For Carl.칼 세이건으로 시작해서 칼 세이건으로 끝난다.,1,칼 세이건으로 시작해서 칼 세이건으로 끝난다,"([(칼, NNG, 0, 1), (세, MM, 2, 1), (이, VCP, 3, 1..."
149994,9458520,디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요.,1,디케이드 다음에 더블 다음에 오즈인데 더블은 조금밖에 안나오네요,"([(디케이드, NNP, 0, 4), (다음, NNG, 5, 2), (에, JKB,..."
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0,인간이 문제지 소는 뭔죄인가,"([(인간, NNG, 0, 2), (이, JKS, 2, 1), (문제지, NNG, ..."
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0,이게 뭐요 한국인은 거들먹거리고 필리핀 혼혈은 착하다,"([(이, MM, 0, 1), (게, EC, 1, 1), (뭐, NP, 3, 1),..."


In [33]:
# kiwi의 형태소 분석 결과 중 단어만을 선택하기 위한 함수
def get_morphed_word(wiki_result):
    txt = []
    for i in wiki_result[0]:
        txt.append(i[0])
    # print(txt)
    
    return txt

In [34]:
morphed_txt = get_morphed_word(dedupe_train_df['morphed'][0])
morphed_txt

['아', '더빙', '진짜', '짜증', '나', '네요', '목소리']

In [35]:
dedupe_train_df['morphed_word'] = dedupe_train_df['morphed'].apply(lambda x: get_morphed_word(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
dedupe_train_df.loc[200:220, 'morphed_word']

200                [용, 건담, 시리즈, 중, 에서, 아직, 까지, 도, 최, 고봉]
201    [개콘, 은, 요즘, 갈수록, 코너, 들, 이, 다노잼, 이, 고, 웃음, 이, 안...
202                [사다코, 의, 한, 이, 서리, ᆫ, 우물, 펀치, ㅜㅜ, 감동]
203    [후세, 와, 사랑, 하, 게, 되, ᆫ, 결정, 적, 계기, 그, 시간, 이, 표...
204    [새벽, 시간, 에, 하, 는, 일본, 영화, 들, 은, 전부, 개졸작이다, 일본,...
205         [강수연, 의, 나가, 어, 있, 어, 그리고, 최정원, 의, 신음, 시, ᆫ]
206     [아, 정말, 김혜성, 너무, 예쁘, 네요, 이현진, 도, 웃, 는, 거, 정말, 하]
207                                     [가발, 쓰, 고, 싶, 다]
208    [화려, 하, ᆫ, 여정, 이, 인상, 깊, 어요, ㅋㅋ, 재밋, 어요, ㅋㅋ, 배...
209                      [대, 들, 을, 위하, ᆫ, 성, 적, 호기심, 영화]
210                                    [감동, 적, 이, ᆫ, 영화]
211            [이거, ᆫ, 말, 이, 필요, 없, 다, 그냥, 닥치, 고, 보, 어라]
212             [각기, 다른, 사람, 들, 의, 재밌, 고, 멋지, ᆫ, 사랑, 영화]
213    [역시, 미국, 드라마, 의, 파워, 정말, 알, ᆯ, 수, 없, 는, 그, 미묘함...
214                                    [너무너무, 훈훈, 하, 네요]
215    [이거, 응답하라, 에서, 은지원, 원, 도, 스스로, 욕하, 지, 않, 앗, 나,...
216    [아, 너무, 웃기, 고, 배꼽, 빠지, ᆯ, 뻔하, 었, 네, 나, 의, 컴, 에...
217    [구성, 이, 상당히, 부실, 하, ᆫ, 영

In [37]:
stopwords=['가', '걍', '과', 'ᆫ', 'ᆫ가', 'ᆫ다', 'ᆫ지', '는', '다', '도', '들', '등', 'ᆯ', '를', '보', '수', '아', '었', '에', '에서', '와', '으로', '은', '을', '의', '이', '있'
           '자', '잘', '좀', '하', '하다','한']

In [38]:
morphed_txt_2 = dedupe_train_df['morphed_word'][219]
morphed_txt_2

['언제', '적', '영웅본색', '연출', '이', 'ᆫ지', '현실', '성', '제로', '이', 'ᆫ', '영화']

In [39]:
[w for w in morphed_txt_2 if not w in stopwords]

['언제', '적', '영웅본색', '연출', '현실', '성', '제로', '영화']

In [40]:
dedupe_train_df['no_stopwords'] = dedupe_train_df['morphed_word'].apply(lambda x: [w for w in x if not w in stopwords])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [41]:
type(dedupe_train_df['no_stopwords'][0])

list

In [42]:
dedupe_train_df['no_stopwords'][0]

['더빙', '진짜', '짜증', '나', '네요', '목소리']

---
### 6) 토큰화(Tokenization)

In [43]:
train_x = dedupe_train_df['no_stopwords'] # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
# train_x = dedupe_train_df['no_stopwords'].apply(lambda x: ' '.join(x)) # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
train_y = dedupe_train_df['label']

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing import sequence

---
(가) 단어 사전을 만들기 위해 첫 번째 토큰화 작업
  - 디폴트 값으로 토큰화하여 결과를 관찰하고 참고 한다.

In [45]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_x)  # fit_on_texts()안에 코퍼스를 입력으로 하면 빈도수를 기준으로 단어 집합(vocabulary)을 생성한다.

In [46]:
# 분석한 결과를 보기 : 전체 어휘의 숫자와 희귀단어 분석

threshold = 3 # 등장 빈도 하한, 이 수보다 작은 빈도를 나타내는 어휘는 희귀 단어로 정한다.
total_cnt = len(tokenizer.word_index) # 단어의 수
rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

단어 집합(vocabulary)의 크기 : 57728
등장 빈도가 2번 이하인 희귀 단어의 수: 40531
단어 집합에서 희귀 단어의 비율: 70.21029656319291
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 2.626372726476925


In [47]:
# print(tokenizer.word_index)
# print(tokenizer.word_counts)

In [48]:
# 전체 어휘 개수 중 빈도수 2이하인 단어 개수는 제거.
# 0번 패딩 토큰과 1번 OOV 토큰을 고려하여 +2

vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :',vocab_size)

단어 집합의 크기 : 17199


---
(나) 앞에서 만든 단어 사전을 이용하여 새롭게 토큰화
  - Tokenizer()를 초기화하고 다시 토큰화 한다.

In [49]:
# vocab_size = 17199

oov_tok = "<OOV>" # Out-Of-Vocabulary 토큰

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_x)

  - 설정값을 바꾸고 다시 토큰화해도 전체 어휘의 수는 변하지 않는다. 인코딩할 때 사용할 어휘 기준만 달라진다.

In [50]:
len(tokenizer.word_counts) # 사용된 전체 어휘의 수, 토큰의 수
# print(tokenizer.word_index)

57728

In [51]:
# type(tokenizer.word_counts) # collections.OrderedDict : 입력된 순서대로 값을 가지고 있는 딕셔너리

In [52]:
# tokenizer.word_counts['영화'] # '영화'란 어휘가 몇 번 사용되었는지 출력
# sorted(tokenizer.word_counts.items()) # 키 값으로 정렬

In [53]:
# 밸류로 정렬, 사용 빈도가 많은 어휘부터 볼 수 있다.
import operator

sorted_dict = sorted(tokenizer.word_counts.items(), key=operator.itemgetter(1), reverse=True)
sorted_dict

[('영화', 57322),
 ('고', 47034),
 ('어', 35085),
 ('게', 25722),
 ('지', 22519),
 ('나', 20258),
 ('있', 16678),
 ('없', 16045),
 ('거', 13847),
 ('되', 13761),
 ('어서', 12916),
 ('만', 11630),
 ('는데', 11512),
 ('너무', 11198),
 ('좋', 11144),
 ('주', 11049),
 ('어요', 10906),
 ('기', 10284),
 ('것', 10094),
 ('적', 10083),
 ('음', 9930),
 ('로', 9757),
 ('안', 9745),
 ('정말', 9654),
 ('ᆷ', 9024),
 ('지만', 8606),
 ('같', 8418),
 ('재밌', 8401),
 ('네요', 8276),
 ('진짜', 8176),
 ('점', 7797),
 ('말', 7719),
 ('아니', 7667),
 ('않', 7616),
 ('네', 7443),
 ('면', 7399),
 ('만들', 7398),
 ('연기', 7199),
 ('나오', 6672),
 ('ᆸ니다', 6439),
 ('뭐', 6123),
 ('이거', 6084),
 ('평점', 6041),
 ('던', 5975),
 ('최고', 5959),
 ('이런', 5933),
 ('시', 5907),
 ('그', 5639),
 ('생각', 5574),
 ('겠', 5559),
 ('스토리', 5327),
 ('왜', 5291),
 ('듯', 5218),
 ('습니다', 5208),
 ('때', 5114),
 ('드라마', 5071),
 ('사람', 5032),
 ('싶', 4966),
 ('더', 4814),
 ('배우', 4808),
 ('보다', 4788),
 ('감동', 4752),
 ('까지', 4604),
 ('어도', 4557),
 ('라', 4387),
 ('내', 4336),
 ('ᆫ데', 4317),
 ('ㅋㅋ', 

In [54]:
tokenizer.word_index # 토큰화 될 때, 사용 빈도에 따라 인덱스 값이 주어진 것을 확인. 

{'<OOV>': 1,
 '영화': 2,
 '고': 3,
 '어': 4,
 '게': 5,
 '지': 6,
 '나': 7,
 '있': 8,
 '없': 9,
 '거': 10,
 '되': 11,
 '어서': 12,
 '만': 13,
 '는데': 14,
 '너무': 15,
 '좋': 16,
 '주': 17,
 '어요': 18,
 '기': 19,
 '것': 20,
 '적': 21,
 '음': 22,
 '로': 23,
 '안': 24,
 '정말': 25,
 'ᆷ': 26,
 '지만': 27,
 '같': 28,
 '재밌': 29,
 '네요': 30,
 '진짜': 31,
 '점': 32,
 '말': 33,
 '아니': 34,
 '않': 35,
 '네': 36,
 '면': 37,
 '만들': 38,
 '연기': 39,
 '나오': 40,
 'ᆸ니다': 41,
 '뭐': 42,
 '이거': 43,
 '평점': 44,
 '던': 45,
 '최고': 46,
 '이런': 47,
 '시': 48,
 '그': 49,
 '생각': 50,
 '겠': 51,
 '스토리': 52,
 '왜': 53,
 '듯': 54,
 '습니다': 55,
 '때': 56,
 '드라마': 57,
 '사람': 58,
 '싶': 59,
 '더': 60,
 '배우': 61,
 '보다': 62,
 '감동': 63,
 '까지': 64,
 '어도': 65,
 '라': 66,
 '내': 67,
 'ᆫ데': 68,
 'ㅋㅋ': 69,
 '재미': 70,
 '알': 71,
 '내용': 72,
 '아깝': 73,
 '감독': 74,
 '그냥': 75,
 '시간': 76,
 '중': 77,
 '어야': 78,
 '재미있': 79,
 '지루': 80,
 '못': 81,
 '쓰레기': 82,
 '사랑': 83,
 '냐': 84,
 '라고': 85,
 '다시': 86,
 '모르': 87,
 '면서': 88,
 '니': 89,
 '작품': 90,
 '보이': 91,
 '야': 92,
 '번': 93,
 '재미없': 94,
 '마지막': 9

---
(다) 토큰화 정보를 파일로 저장한다.
  - 이렇게 저장하면 필요할 때 읽어서 사용할 수 있다. 특히 추론할 때 잘 사용할 수 있다.

In [55]:
# tokenizer.get_config() # 설정값과 토큰화 한 정보를 볼 수 있다.

# 토큰화 정보를 json 형식으로 파일에 저장
# DATA_DIR = 'data'
TOKENIZED_FILE = 'tokenized.json'
TOKENIZED_PATH = os.path.join(DATA_DIR, TOKENIZED_FILE)

import json

tok_json = tokenizer.to_json()
# with io.open(DATA_IN_DIR + 'tokenizer_' + ver + '.json', 'w', encoding='utf-8') as f:
#     f.write(json.dumps(tokenizer_json, ensure_ascii=False))

json.dump(tok_json, open(TOKENIZED_PATH, 'w'), ensure_ascii=False)

# 읽는 방법 : 아래 2 가지 방법 중 한 가지 방법으로 읽어 올 수 있다.
'''
tok_configs = None
with open(tokenized_file, 'r') as f:
    tok_configs = json.load(f)
'''
# keras.preprocessing.text.tokenizer_from_json(json_string)
'''
with open('tokenizer.json') as f:
    data = json.load(f)
    tokenizer = tokenizer_from_json(data)
'''

"\nwith open('tokenizer.json') as f:\n    data = json.load(f)\n    tokenizer = tokenizer_from_json(data)\n"

---
### 7) 시퀀싱과 패딩(Sequencing and Padding)  
토큰화 된 데이터를 숫자(인덱스) 시퀀스로 바꾸고 같은 길이의 시퀀스로 만들기 위해 패딩을 한다.

In [56]:
# 문자열 리스트로 이루어진 입력데이터를 숫자 시퀀스로 변경한다. 
training_sequences = tokenizer.texts_to_sequences(train_x)

In [57]:
# training_sequences
training_sequences[0]

[539, 31, 197, 7, 30, 649]

In [58]:
text = tokenizer.sequences_to_texts([training_sequences[0]]) # 역으로 문장 확인
text

['더빙 진짜 짜증 나 네요 목소리']

In [59]:
# Defining pre-processing hyperparameters
max_len = 30 # 시퀀스의 길이를 30으로 고정
trunc_type = "post" # 길이가 30 보다 길 때 뒷 부분을 버린다. 
padding_type = "post" # 길이가 30 보다 짧을 대 뒷 부분을 0으로 채운다.

# padding : 뒤를 0으로 채운다.
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [60]:
training_sequences[0] # 확인

[539, 31, 197, 7, 30, 649]

In [61]:
train_padded[0]

array([539,  31, 197,   7,  30, 649,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0], dtype=int32)

In [62]:
text_paddes = tokenizer.sequences_to_texts([train_padded[0]]) # 역으로 문장 확인
text_paddes

['더빙 진짜 짜증 나 네요 목소리 <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV> <OOV>']

In [63]:
# Shape of train tensor
print('Shape of training tensor: ', train_padded.shape)

Shape of training tensor:  (143460, 30)


In [64]:
# Before padding
len(training_sequences[0]), len(training_sequences[1])

(6, 9)

In [65]:
# After padding
len(train_padded[0]), len(train_padded[1])

(30, 30)

---
### 8) 전처리(PreProcessed)된 Data를 파일에 저장

dataframe의 객체를 그대로 저장하려면, (list, dict 등 포함) pickle로 저장한다.    
to_pickle, read_pickle도 사용 가능 :   
https://wikidocs.net/8929  
https://tariat.tistory.com/739  

In [66]:
# DATA_DIR = 'data'
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')
os.makedirs(PROCESSED_DATA_DIR, exist_ok=True)

PROCESSED_DATA_FILE = "train_padded.p"
PROCESSED_LABEL_FILE = "train_label.p"

PROCESSED_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_DATA_FILE)
PROCESSED_LABEL_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_LABEL_FILE)

In [67]:
train_y

0         0
1         1
2         0
3         0
4         1
         ..
149993    1
149994    1
149995    0
149997    0
149998    1
Name: label, Length: 143460, dtype: int64

In [68]:
# 데이터 저장

import pickle

with open(PROCESSED_DATA_PATH, "wb") as file:
    pickle.dump(train_padded, file)
    
with open(PROCESSED_LABEL_PATH, "wb") as file:
    pickle.dump(train_y, file)

In [69]:
# 저장된 파일 로드 확인
with open(PROCESSED_DATA_PATH, "rb" ) as file:
    train_padded_loaded = pickle.load(file)

In [70]:
train_padded_loaded

array([[  539,    31,   197, ...,     0,     0,     0],
       [  875,   453,     3, ...,     0,     0,     0],
       [    1,   545,    20, ...,     0,     0,     0],
       ...,
       [  223, 11392,   632, ...,     0,     0,     0],
       [    5,    42,   124, ...,     0,     0,     0],
       [  922,     2,   860, ...,     0,     0,     0]], dtype=int32)

---
### 9) Test dataset 전처리

트레이닝 후 정확도를 측정하기 위해 사용하기 위해 테스트 데이터셋도 전처리해 둔다.
  1. test dataset load 
  2. 결측 데이터 제거
  3. 한글만 추출
  4. 형태소 분석
  5. 토큰화
  6. 시퀀스 만들기와 패딩

In [71]:
TEST_DATA_FILE = 'ratings_test.txt'
TEST_DATA_PATH = os.path.join(DATA_DIR, TEST_DATA_FILE)

In [72]:
test_df = pd.read_table(TEST_DATA_PATH)

In [73]:
test_no_nan_df = test_df.dropna(axis=0) # 결측 데이터 제거
# test_no_nan_df

In [74]:
test_no_nan_df['document_kor'] = test_no_nan_df['document'].apply(lambda x: hangul.sub('', x).strip()).copy() # 빈 칸이 하나 남는 것을 없애기 위해 .strip()을 사용
test_no_nan_df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,document,label,document_kor
0,6270596,굳 ㅋ,1,굳 ㅋ
1,9274899,GDNTOPCLASSINTHECLUB,0,
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠


In [75]:
test_no_nan_df['document_kor'].replace('', np.nan, inplace=True) # 한글만 남긴 데이터에서 빈 문자열을 NaN값으로 변환한다. 이렇게 하여야 삭제하기 쉽다.

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [76]:
print('null data가 있는가? : ', test_no_nan_df.isnull().values.any())
print('----------')
print('null data 건수 \n', '-----\n', test_no_nan_df.isnull().sum())

null data가 있는가? :  True
----------
null data 건수 
 -----
 id                0
document          0
label             0
document_kor    422
dtype: int64


In [77]:
test_no_nan_df.shape

(49997, 4)

In [78]:
test_no_nan_df = test_no_nan_df.dropna(axis=0) # nan이 있는 모든 row를 없애는 코드

In [79]:
test_no_nan_df.shape

(49575, 4)

In [80]:
test_no_nan_df['document_kor'].nunique()

48342

In [81]:
# 중복된 데이터 확인
test_no_nan_df[test_no_nan_df.duplicated('document_kor')]

,id,document,label,document_kor
599,6983606,재미있어요~~,1,재미있어요
757,3384300,별로다...,0,별로다
853,4528041,최고,1,최고
956,195683,재미있어요,1,재미있어요
1089,2235528,볼만함,1,볼만함
...,...,...,...,...
49775,632708,괜찮음..,1,괜찮음
49841,8097768,2년이 지나도 잊혀지지 않는 소중한 드라마.,1,년이 지나도 잊혀지지 않는 소중한 드라마
49923,4540652,재미있게 봤습니다!,1,재미있게 봤습니다
49930,7500092,ㅠ~~ㅠ,0,ㅠㅠ


In [82]:
# document_kor 열에서 중복인 내용이 있다면 중복 제거
test_no_nan_df = test_no_nan_df.drop_duplicates('document_kor', keep='first')

In [83]:
test_no_nan_df.shape

(48342, 4)

In [84]:
start = time.time() # start time

test_no_nan_df['morphed'] = test_no_nan_df['document_kor'].apply(lambda x: kiwi.analyze(x)[0]) # 40 분 정도 걸림. # data 중에 형식이 맞지 않는 데이터가 입력되면 에러가 난다. NaN이 있는 경우
# df_new = dedupe_kor_df.apply(lambda x: (kiwi.analyze(x))[0])

end = time.time() # end time

print("time : ", end - start, " sec") # sec

time :  39.60696196556091  sec


In [85]:
test_no_nan_df

,id,document,label,document_kor,morphed
0,6270596,굳 ㅋ,1,굳 ㅋ,"([(굳, VV, 0, 1), (ㅋ, SW, 2, 1)], -26.845458984..."
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,"([(뭐, NP, 0, 1), (이, VCP, 1, 1), (야, EF, 1, 1)..."
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,"([(지루, XR, 0, 2), (하, XSA, 2, 1), (지, EC, 3, 1..."
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,"([(만, NNP, 0, 1), (아니, VCN, 2, 2), (었, EP, 4, ..."
5,7898805,"음악이 주가 된, 최고의 음악영화",1,음악이 주가 된 최고의 음악영화,"([(음악, NNG, 0, 2), (이, JKS, 2, 1), (주, NNG, 4,..."
...,...,...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,"([(오랜만, NNG, 0, 3), (에, JKB, 3, 1), (평점, NNG, ..."
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,"([(의지, NNG, 0, 2), (박약, NNG, 3, 2), (들, XSN, 5..."
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,"([(그림, NNG, 0, 2), (도, JX, 2, 1), (좋, VA, 4, 1..."
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,"([(절대, MAG, 0, 2), (보, VV, 4, 1), (어서, EC, 3, ..."


In [86]:
test_no_nan_df['morphed_word'] = test_no_nan_df['morphed'].apply(lambda x: get_morphed_word(x))

In [87]:
test_no_nan_df

,id,document,label,document_kor,morphed,morphed_word
0,6270596,굳 ㅋ,1,굳 ㅋ,"([(굳, VV, 0, 1), (ㅋ, SW, 2, 1)], -26.845458984...","[굳, ㅋ]"
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,"([(뭐, NP, 0, 1), (이, VCP, 1, 1), (야, EF, 1, 1)...","[뭐, 이, 야, 이, 평점, 들, 은, 나쁘, 지, ᆫ, 않, 지만, 점, 짜리,..."
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0,지루하지는 않은데 완전 막장임 돈주고 보기에는,"([(지루, XR, 0, 2), (하, XSA, 2, 1), (지, EC, 3, 1...","[지루, 하, 지, 는, 않, 은데, 완전, 막장, 임, 돈, 주, 고, 보, 기,..."
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,"([(만, NNP, 0, 1), (아니, VCN, 2, 2), (었, EP, 4, ...","[만, 아니, 었, 어도, 별, 다섯, 개, 주, 었, 을, 터, 이, ᆫ데, 왜,..."
5,7898805,"음악이 주가 된, 최고의 음악영화",1,음악이 주가 된 최고의 음악영화,"([(음악, NNG, 0, 2), (이, JKS, 2, 1), (주, NNG, 4,...","[음악, 이, 주, 가, 되, ᆫ, 최고, 의, 음악, 영화]"
...,...,...,...,...,...,...
49995,4608761,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,1,오랜만에 평점 로긴했네ㅋㅋ 킹왕짱 쌈뽕한 영화를 만났습니다 강렬하게 육쾌함,"([(오랜만, NNG, 0, 3), (에, JKB, 3, 1), (평점, NNG, ...","[오랜만, 에, 평점, 로, 기, ᆫ, 하, 었, 네, ㅋㅋ, 킹왕짱, 쌈뽕, 하,..."
49996,5308387,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따 OOOO,0,의지 박약들이나 하는거다 탈영은 일단 주인공 김대희 닮았고 이등병 찐따,"([(의지, NNG, 0, 2), (박약, NNG, 3, 2), (들, XSN, 5...","[의지, 박약, 들, 이나, 하, 는, 거, 다, 탈영, 은, 일단, 주인공, 김대..."
49997,9072549,그림도 좋고 완성도도 높았지만... 보는 내내 불안하게 만든다,0,그림도 좋고 완성도도 높았지만 보는 내내 불안하게 만든다,"([(그림, NNG, 0, 2), (도, JX, 2, 1), (좋, VA, 4, 1...","[그림, 도, 좋, 고, 완성도, 도, 높, 었, 지만, 보, 는, 내내, 불안, ..."
49998,5802125,절대 봐서는 안 될 영화.. 재미도 없고 기분만 잡치고.. 한 세트장에서 다 해먹네,0,절대 봐서는 안 될 영화 재미도 없고 기분만 잡치고 한 세트장에서 다 해먹네,"([(절대, MAG, 0, 2), (보, VV, 4, 1), (어서, EC, 3, ...","[절대, 보, 어서, 는, 안, 되, ᆯ, 영화, 재미, 도, 없, 고, 기분, 만..."


In [88]:
test_x = test_no_nan_df['morphed_word'] # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
# train_x = dedupe_train_df['no_stopwords'].apply(lambda x: ' '.join(x)) # tokenizer에 입력할 데이터를 준비, tokenizer는 리스트와 문자열 모두 입력할 수 있다.
test_y = test_no_nan_df['label']

In [89]:
test_sequences = tokenizer.texts_to_sequences(test_x)

In [90]:
test_padded = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

In [91]:
# Shape of test tensor
print('Shape of test tensor: ', test_padded.shape)

# testing_sequences = tokenizer.texts_to_sequences(test_x)
# testing_padded = pad_sequences(testing_sequences, maxlen = max_len, padding = padding_type, truncating = trunc_type)

Shape of test tensor:  (48342, 30)


In [92]:
PROCESSED_TEST_DATA_FILE = "test_padded.p"
PROCESSED_TEST_LABEL_FILE = "test_label.p"

PROCESSED_TEST_DATA_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_TEST_DATA_FILE)
PROCESSED_TEST_LABEL_PATH = os.path.join(PROCESSED_DATA_DIR, PROCESSED_TEST_LABEL_FILE)

In [93]:
# 데이터 저장
with open(PROCESSED_TEST_DATA_PATH, "wb") as file:
    pickle.dump(test_padded, file)
    
with open(PROCESSED_TEST_LABEL_PATH, "wb") as file:
    pickle.dump(test_y, file)